# [662. Maximum Width of Binary Tree](https://leetcode.com/problems/maximum-width-of-binary-tree/)

Given a binary tree, write a function to get the maximum width of the given tree. The width of a tree is the maximum width among all levels. The binary tree has the same structure as a full binary tree, but some nodes are null.

The width of one level is defined as the length between the end-nodes (the leftmost and right most non-null nodes in the level, where the null nodes between the end-nodes are also counted into the length calculation.

## 自己版

我发现的规律是这样的：当某个节点最左边的节点和最右边的节点都存在时，计算该节点的高度，则要求的最大宽度为2**最大高度值，遗憾的是我并没有写出来。。。几个测试用例都跑不通。然后看了下他人的答案，核心是依托 满二叉树 的特性，计算这层的宽度时就是 R-L+1;

```
    from collections import defaultdict
    
    def widthOfBinaryTree(self, root: TreeNode) -> int:
        left = defaultdict(list)
        right = defaultdict(list)
        
        def dfs(node,arrow,level,pos):
            if node:            
                if arrow == 'l':
                    left[level].append(pos)
                if arrow == 'r':
                    right[level].append(pos)

                dfs(node.left,'l',level+1,pos*2)
                dfs(node.right,'r',level+1,pos*2+1)
            
        dfs(root,None,0,1)
        print('l',left)
        print('r',right)
```

打印的结果：

```
l defaultdict(<class 'list'>, {1: [2], 2: [4]})
r defaultdict(<class 'list'>, {2: [5, 7], 1: [3]})
```

这样，left 和 right 里存储的就是 以节点层数为key，value 为满二叉树时对应的数值，然后再按层数遍历，对于相同层数有多个value值的情况，l 只取最小值，r 只需最大值，这样 r-l+1 就可以得出最终最大的结果。很显然这样先求出整体情况，再进行遍历，无论从事件复杂度上看还是空间复杂度上看，都是可以进一步优化的。

## DFS 版

```
    def widthOfBinaryTree(self, root: TreeNode) -> int:
        left = {} # 不需要记录 right 
        self.res = 0
        def dfs(node,depth,pos):
            if node:
                left.setdefault(depth,pos) # 只会记录当前层左子树最左边的数据
                self.res = max(self.res, pos-left[depth]+1)
                dfs(node.left,depth+1,pos*2)
                dfs(node.right,depth+1,pos*2+1)
        dfs(root,0,1)
        return self.res
```

<font color=red>
除了上面使用 setdefault(depth,pos) 方式来记录某一层最左边的节点值，还可以用什么方法呢？
</font>

```
    def widthOfBinaryTree(self, root: TreeNode) -> int:
        left = {} # 不需要记录 right 
        self.res = 0
        def dfs(node,depth,pos):
            if node:
                # 只需要记录当前层左子树最左边的数据
                if len(left) == depth:
                    left[depth] = pos
                self.res = max(self.res, pos-left[depth]+1)
                dfs(node.left,depth+1,pos*2)
                dfs(node.right,depth+1,pos*2+1)
        dfs(root,0,1)
        return self.res
```

为什么这么做可以呢？因为除了根节点那层，每一层只需要记录一个数据，且结构为 {depth:index} 形式，这样 left 的长度就和 depth 建立了联系。

## BFS

```
    def widthOfBinaryTree(self, root: TreeNode) -> int:
        # [(node,depth,index)]
        queue = [(root,0,1)]
        left = {}
        res = 0
        
        while len(queue) > 0:
            size = len(queue)
            for i in range(size):
                cur,depth,index = queue.pop(0)
                
                if cur.left:
                    queue.append((cur.left, depth+1,index*2))
                    
                if cur.right:
                    queue.append((cur.right, depth+1, index*2+1))
                    
                if depth == len(left):
                    left[depth] = index
                res = max(res, index-left[depth]+1)
        
        return res
```

## 更高效版

```
    def widthOfBinaryTree(self, root: TreeNode) -> int:
        queue = [(root, 0, 0)]
        cur_depth = left = ans = 0
        for node, depth, pos in queue:
            if node:
                queue.append((node.left, depth+1, pos*2))
                queue.append((node.right, depth+1, pos*2 + 1))
                if cur_depth != depth: # 从这里开始注意一下
                    cur_depth = depth
                    left = pos
                ans = max(pos - left + 1, ans)
        return ans
```

<font color=red>比上个版本少了层数的存储：</font>

```
    def widthOfBinaryTree(self, root: TreeNode) -> int:
        queue = [(root,0)]
        res = 0
        while queue:
            arr = []
            for _ in range(len(queue)):
                node,pos = queue.pop(0)
                arr.append(pos)
                if node.left:
                    queue.append((node.left,pos*2))
                if node.right:
                    queue.append((node.right,pos*2+1))
            
            res = max(res,1+arr[-1]-arr[0])
        
        return res
```